In [1]:
import pandas 
from os import listdir
from os.path import isfile, join

def limpaDf(df):
        new = df[ ["Ação Orçamentária", "Município", "Valor Transferido"]].copy() 
        return new 
        

In [41]:

path = r"C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016"
directory = listdir(path)
all_data_frames = []


for file in directory:
    file_path = path + "\\" + file
    print(file_path)
    df = pandas.read_csv( file_path ,  delimiter=";", encoding="utf-8-sig")
    df = limpaDf(df)
    all_data_frames.append(df)

acoes2016 = pandas.concat(all_data_frames)


C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 00PI 2016 1.csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 00PI 2016 2 (jul - set).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 00PI 2016 3 (out - dez).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0509 2016.csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0515  2016 1 (jan - jun).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0515  2016 2 (jul - dez).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0969 2016 1 (jan - jun).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0969 2016 2 (jul - set).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0969 2016 3 (out - dez).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0c33 2016 1 (jan - abr).csv
C:\Users\aaron\Documents\Code\pibit\dados\Transferencias 2016\acao 0c33 20

In [43]:
acoes2016.to_csv(r"C:\Users\aaron\Documents\Code\pibit\dados\dados limpos\acoes2016.csv")

In [7]:
file_path =  r"C:\Users\aaron\Documents\Code\pibit\dados\dados limpos\acoes2016.csv"
acoes = pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig")

In [8]:
import pandasql as ps

municipios = acoes[["Município"]].copy()

municipios = municipios.drop_duplicates(subset="Município")

In [11]:
municipios_list = list(municipios["Município"])

for municipio in municipios_list :
    query = "select * from acoes where Município =  '" + municipio + "'"
    df_municipio = ps.sqldf(query, locals())
    
    df_municipio.to_csv(r"C:\Users\aaron\Documents\Code\pibit\dados\dados limpos\acoes-municipio\\" + municipio + ".csv")
